In [1]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
from src.data_loader import GetDataLoaders, imshow

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_dir = '..\\data\\miniImageNet'
dataloaders, class_names, dataset_sizes  = GetDataLoaders(data_dir)
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

FileNotFoundError: Found no valid file for the classes n01749939, n01770081, n01843383, n01910747, n02074367, n02089867, n02091831, n02108089, n02108551, n02108915, n02111277, n02113712, n02120079, n02165456, n02457408, n02606052, n02687172, n02747177, n02795169, n02823428, n02966193, n03017168, n03047690, n03062245, n03207743, n03220513, n03337140, n03347037, n03400231, n03476684, n03527444, n03676483, n03838899, n03854065, n03888605, n03908618, n03924679, n03998194, n04067472, n04243546, n04251144, n04258138, n04275548, n04296562, n04389033, n04435653, n04443257, n04509417, n04515003, n04596742, n04604644, n04612504, n06794110, n07584110, n07697537, n07747607, n09246464, n13054560, n13133613. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

In [ ]:
inputs, classes = next(iter(dataloaders['val']))
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

In [ ]:
inputs, classes = next(iter(dataloaders['test']))
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])